In [3]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    return angle

In [3]:
JointColours = [100,100,255,100,255,100]

RightShoulderCorrect = 0
RightElbowCorrect = 0
LeftElbowCorrect = 0
LeftShoulderCorrect = 0

In [4]:
def Pose1(ElbowLAngle, ShoulderLAngle, ElbowRAngle, ShoulderRAngle):
    global LeftElbowCorrect, LeftShoulderCorrect, RightShoulderCorrect, RightElbowCorrect

    LeftElbowCorrect = 3 if 150 < ElbowLAngle < 190 else 0
    LeftShoulderCorrect = 3 if 110 < ShoulderLAngle < 150 else 0
    RightElbowCorrect = 3 if 150 < ElbowRAngle < 190 else 0
    RightShoulderCorrect = 3 if 110 < ShoulderRAngle <150 else 0

In [5]:
def Pose2(ElbowLAngle, ShoulderLAngle, ElbowRAngle, ShoulderRAngle):
    global LeftElbowCorrect, LeftShoulderCorrect, RightShoulderCorrect, RightElbowCorrect

    LeftElbowCorrect = 3 if 20 < ElbowLAngle < 50 else 0
    LeftShoulderCorrect = 3 if 90 < ShoulderLAngle < 110 else 0
    RightElbowCorrect = 3 if 20 < ElbowRAngle < 50 else 0
    RightShoulderCorrect = 3 if 90 < ShoulderRAngle <110 else 0

In [22]:
def Pose3(ElbowLAngle, ShoulderLAngle, ElbowRAngle, ShoulderRAngle):
    global LeftElbowCorrect, LeftShoulderCorrect, RightShoulderCorrect, RightElbowCorrect

    LeftElbowCorrect = 3 if 20 < ElbowLAngle < 50 else 0
    LeftShoulderCorrect = 3 if 90 < ShoulderLAngle < 110 else 0
    RightElbowCorrect = 3 if 20 < ElbowRAngle < 50 else 0
    RightShoulderCorrect = 3 if 90 < ShoulderRAngle <110 else 0

In [6]:
PoseChoice = 1

In [7]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulderL = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbowL = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wristL = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            shoulderR = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbowR = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wristR = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            Face = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y]  
            # Calculate angle
            ElbowLAngle = calculate_angle(shoulderL, elbowL, wristL)
            
            ElbowRAngle = calculate_angle(shoulderR, elbowR, wristR)

            ShoulderRAngle = calculate_angle(shoulderL, shoulderR, elbowR)
            
            ShoulderLAngle = calculate_angle(shoulderR, shoulderL, elbowL)
            

            
            #Convert angle into integer
            ElbowLAngle = int(ElbowLAngle)
            ElbowRAngle = int(ElbowRAngle)
            ShoulderRAngle = int(ShoulderRAngle)
            ShoulderLAngle = int(ShoulderLAngle)


            match PoseChoice:
                case 1:
                    Pose1(ElbowLAngle, ShoulderLAngle, ElbowRAngle, ShoulderRAngle)
                case 2:
                    Pose2(ElbowLAngle, ShoulderLAngle, ElbowRAngle, ShoulderRAngle)

            
            
            
            # Visualize angle
            cv2.putText(image, str(ElbowLAngle), 
                           tuple(np.multiply(elbowL, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (JointColours[LeftElbowCorrect], JointColours[LeftElbowCorrect+1], JointColours[LeftElbowCorrect+2]), 2, cv2.LINE_AA
                                )

            cv2.putText(image, str(ElbowRAngle), 
                           tuple(np.multiply(elbowR, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (JointColours[RightElbowCorrect], JointColours[RightElbowCorrect+1],JointColours[RightElbowCorrect+2]), 2, cv2.LINE_AA
                                )

            cv2.putText(image, str(ShoulderRAngle), 
                           tuple(np.multiply(shoulderR, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (JointColours[RightShoulderCorrect], JointColours[RightShoulderCorrect+1], JointColours[RightShoulderCorrect+2]), 2, cv2.LINE_AA
                                )

            cv2.putText(image, str(ShoulderLAngle), 
                           tuple(np.multiply(shoulderL, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (JointColours[LeftShoulderCorrect], JointColours[LeftShoulderCorrect+1], JointColours[LeftShoulderCorrect+2]), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()